в общем, необходимый функционал готов. осталось добавить предварительную обработку изображения и мб улучшить дизайн (но в pyqt5 это проблематично)

In [1]:
import os
import cv2
import sys
from PyQt5 import QtWidgets
from PyQt5.QtCore import Qt
from PyQt5.uic import loadUi
from PyQt5.QtGui import QPixmap, QPixmapCache, QImage
from PyQt5.QtWidgets import QDialog, QApplication, QWidget, QMainWindow, QFileDialog

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

def to_gray_image(filepath):
    image = cv2.imread(filepath)
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def to_gray_folder(filepath):
    images = load_images_from_folder(filepath)
    images_gray = []
    for image in images:
        images_gray.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
    return images_gray

class WelcomeScreen(QMainWindow):
    def __init__(self):
        super(WelcomeScreen,self).__init__()
        loadUi("page1.ui",self)
        pixmap = QPixmap('logo(2).png')
        self.label.setPixmap(pixmap)
        self.start_btn.clicked.connect(lambda: self.start_click())
    
    def start_click(self):
        select_file = SelectScreen() 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)

class SelectScreen(QMainWindow):
    def __init__(self):
        super(SelectScreen,self).__init__()
        loadUi("page2.ui",self)
        self.setAcceptDrops(True)
        self.pushFolderButton.clicked.connect(lambda: self.open_folder())
        self.pushImageButton.clicked.connect(lambda: self.open_image())
    
    def open_folder(self):
        file = QtWidgets.QFileDialog.getExistingDirectory(self, 'Select Folder')
        if os.path.exists(file):
            select_file = FolderPretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)
        
    def open_image(self):
        file = QtWidgets.QFileDialog.getOpenFileName(self, "Select Image", None, "Image (*.png *.jpg)")[0]
        if "jpg" in file or "png" in file:
            select_file = ImagePretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)
        
    def dragEnterEvent(self, event):
        if event.mimeData().hasImage:
            event.accept()
        else:
            event.ignore()

    def dragMoveEvent(self, event):
        if event.mimeData().hasImage:
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().hasImage:
            event.setDropAction(Qt.CopyAction)
            self.file_path = event.mimeData().urls()[0].toLocalFile()
            self.set_image(self.file_path)
            event.accept()
        else:
            event.ignore()

    def set_image(self, file):
        if "jpg" in file or "png" in file:
            select_file = ImagePretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)
        else:
            select_file = FolderPretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)

class ImagePretrainScreen(QMainWindow):
    def __init__(self, file):
        super(ImagePretrainScreen,self).__init__()
        loadUi("page3_image.ui",self)
        pixmap = QPixmap(file)
        pixmap= pixmap.scaledToWidth((cv2.imread(file).shape[:2][1])%450)
        self.label.setPixmap(pixmap)
        print(file)
        
        self.toNeuralButton.clicked.connect(lambda: self.findDrops(file))
        self.ImageProcessingButton.clicked.connect(lambda: self.ImageProcessing(file))
        self.BackButton.clicked.connect(lambda: self.Back())
    
    def findDrops(self, file):
        select_file = ImageResultScreen(file) 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)
        
    def ImageProcessing(self, file):
        print("to do")
        
    def Back(self):
        select_file = SelectScreen()
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)

class FolderPretrainScreen(QMainWindow):
    def __init__(self, file):
        super(FolderPretrainScreen,self).__init__()
        loadUi("page3_folder.ui",self)
        self.label.setText("Your choice: \n" + file)
        print(file)
        
        self.toNeuralButton.clicked.connect(lambda: self.findDrops(file))
        self.BackButton.clicked.connect(lambda: self.Back())
    
    def findDrops(self, file):
        select_file = FolderResultScreen(file) 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)
        
    def Back(self):
        select_file = SelectScreen() 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)


class ImageResultScreen(QMainWindow):
    def __init__(self, file):
        super(ImageResultScreen,self).__init__()
        loadUi("page5_image.ui",self)
        result_img = to_gray_image(file)
        
        if len(result_img.shape)<3:
            frame = cv2.cvtColor(result_img, cv2.COLOR_GRAY2RGB)
        else:
            frame = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
            
        h, w = result_img.shape[:2]
        bytesPerLine = 3 * w
        qimage = QImage(frame.data, w, h, bytesPerLine, QImage.Format.Format_RGB888) 
        pix = QPixmap(qimage)
        self.label.setPixmap(pix.scaledToWidth(w%450))
        
        self.saveButton.clicked.connect(lambda: self.savePhoto(result_img))
        self.quitButton.clicked.connect(lambda: QApplication.closeAllWindows())

    def savePhoto(self, image):
        filename = QFileDialog.getSaveFileName(filter="JPG(*.jpg);;PNG(*.png)")[0]
        cv2.imwrite(filename, image)
        print('Image saved as:',filename)

class FolderResultScreen(QMainWindow):
    def __init__(self, file):
        super(FolderResultScreen,self).__init__()
        loadUi("page5_folder.ui",self)
        result_img = to_gray_folder(file)
        
        self.index = 0
        
        if len(result_img[self.index].shape)<3:
            frame = cv2.cvtColor(result_img[self.index], cv2.COLOR_GRAY2RGB)
        else:
            frame = cv2.cvtColor(result_img[self.index], cv2.COLOR_BGR2RGB)
            
        h, w = result_img[self.index].shape[:2]
        bytesPerLine = 3 * w
        qimage = QImage(frame.data, w, h, bytesPerLine, QImage.Format.Format_RGB888) 
        pix = QPixmap(qimage)
        self.label.setPixmap(pix.scaledToWidth(w%450))
        
        if(len(result_img) == 2):
            self.label_3.setText("and "+str(len(result_img)-1)+" more image")
        if(len(result_img) > 2):
            self.label_3.setText("and "+str(len(result_img)-1)+" more images")
            
        self.saveButton.clicked.connect(lambda: self.savePhoto(result_img))
        self.quitButton.clicked.connect(lambda: QApplication.closeAllWindows())

    def savePhoto(self, images):
        folder = QtWidgets.QFileDialog.getExistingDirectory(self, 'Select Folder')
        print(folder)
        if os.path.exists(folder):
            for i in range(len(images)):
                cv2.imwrite(folder+"/result"+str(i)+".png", images[i])
                print('Image saved as:',folder+"/result"+str(i)+".png")
                
            
# main
app = QApplication(sys.argv)
welcome = WelcomeScreen()
widget = QtWidgets.QStackedWidget()
widget.addWidget(welcome)
widget.setFixedHeight(590)
widget.setFixedWidth(800)
widget.show()
try:
    sys.exit(app.exec_())
except:
    print("Exiting")

C:/Users/USER/Downloads/drop/data
C:/Users/USER/Downloads/drop/test_folder
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result0.png
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result1.png
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result2.png
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result3.png
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result4.png
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result5.png
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result6.png
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result7.png
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result8.png
Image saved as: C:/Users/USER/Downloads/drop/test_folder/result9.png
Exiting
